In [2]:
import pandas as pd
import re
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pyodbc
import uuid


d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pyodbc

# Connection string example
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8JEBTPG;'  # Specify the server name
    'DATABASE=tweets;'  # Specify the database name
    'UID=sa;'  # Specify your username
    'PWD=Triomphe&14;'  # Specify your password
)

# Create a cursor from the connection
cursor = conn.cursor()

# Example: execute a query
cursor.execute("SELECT * FROM tweet")

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print rows
for row in rows:
    print(row)

# Close the connection
conn.close()


In [7]:
df1 = pd.read_csv('tweets_1.csv') 
df1["Text"] = df1["Text"].astype(str)
df1.head()

,Tweet_count,Username,Text,Created At,Retweets,Likes
0,1,England,Our fans. Our players. Our summer.\n\nThis is ...,Thu Jun 06 18:00:20 +0000 2024,4543,26633
1,2,Tommy Robinson 🇬🇧,Man dies at Yorkshire dales Hotspot at the wee...,Tue Aug 13 14:55:10 +0000 2024,1391,9769
2,3,England Extra,For the people mocking Lee Carsley being a pos...,Tue Jul 16 17:25:57 +0000 2024,433,4542
3,4,CentreGoals.,🚨🚨| Manchester United are monitoring Eberechi ...,Tue Sep 10 11:24:17 +0000 2024,80,1817
4,5,England,Ready for a big summer! 📸\n\n#ThreeLions | @ma...,Mon Jun 10 20:24:12 +0000 2024,2011,22827


In [ ]:
df2 = pd.read_csv('tweets_2.csv') 
df2.head()

In [9]:
df2.size

11394

In [10]:
11394/2142

5.319327731092437

In [ ]:
# df2['tweet_date'] = pd.to_datetime(df2['tweet_date'], format='%a %b %d %H:%M:%S %z %Y')
df2['tweet_date'] = df2['tweet_date'].dt.strftime("%Y-%m-%d")


In [ ]:
df1['Created At'] = pd.to_datetime(df1['Created At'], format='%a %b %d %H:%M:%S %z %Y')
df1['Created At'] = df1['Created At'].dt.strftime("%Y-%m-%d %H:%M:%S")
df1 = df1.drop(columns=["Tweet_count" ,"Username", "Retweets", "Likes"])
df1.rename(columns={'Text': 'tweet_content', 'Created At':'tweet_date'}, inplace=True)

In [28]:
df2.head()

,tweet_location,tweet_date,tweet_content
0,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...
1,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...
2,NaN,02-03-2020,Find out how you can protect yourself and love...
3,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...
4,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...


In [19]:
# data cleaning and prep
def data_cleaning_and_preparation(df, text):
    # 1. Drop rows with missing values in text columns
    df.dropna(subset=[text], inplace=True)

    # 2. Remove duplicates
    df.drop_duplicates(subset=[text], inplace=True)

    # 3. Convert text to lowercase
    df[text] = df[text].str.lower()

    # 4. Remove special characters, punctuation, and numimbers
    df[text] = df[text].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

    # 5. Remove extra whitespace
    df[text] = df[text].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # 7. Check the cleaned data
    print(df.head())
    return df
    # Save the cleaned text data
    # df.to_csv('cleaned_text_data.csv', index=False)
    
    
    


In [ ]:
df2 = data_cleaning_and_preparation(df2, text='tweet_content')


In [ ]:
import pandas as pd
import uuid
import numpy as np


# List to store processed DataFrames
processed_dfs = []

# Split the DataFrame into 5 equal parts
split_dfs = np.array_split(df2, 5)

for df_part in split_dfs:
    # Apply sentiment analysis
    sentiment_result = sentiment_analysis_transformers(df_part, text='tweet_content')
    
    # Apply hashtag generator and drop "scores" column
    df_with_hashtags = hashtag_generator(df_part, text_column='tweet_content')
    df_with_hashtags = df_with_hashtags.drop(columns=["scores"])
    
    # Add unique IDs and set empty 'tweet_location'
    df_with_hashtags['id_dist'] = [uuid.uuid4() for _ in range(len(df_with_hashtags))]
    df_with_hashtags['tweet_location'] = ""
    
    # Append the processed DataFrame to the list
    processed_dfs.append(df_with_hashtags)

# Concatenate all processed DataFrames into a single DataFrame
final_df = pd.concat(processed_dfs, ignore_index=True)


In [21]:
def sentiment_analysis_transformers(df, text):
    
    model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
    analyser = pipeline("sentiment-analysis", model=model_path)
    df['scores'] = df[text].apply(lambda text: analyser (text))
    df['Sentiment'] = df['scores'].apply(lambda output: output[0]['label']) 
    df['sentiment_score'] = df['scores'].apply(lambda output: output[0]['score']) 
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 'Positive ❤️' if x == 'positive' else ('Negative 😡' if x == 'negative' else 'Neutral 💛')) 
    df = df.drop(columns = ["scores"])
    return df

In [10]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk


def hashtag_generator(df, text_column):
    nltk.download('punkt')

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-base-tag-generation")
    model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-base-tag-generation")
    
    # Function to generate hashtags for a single text
    def generate_tags(text):
        if not isinstance(text, str) or len(text.strip()) == 0:
            return []  # Return an empty list for empty or missing text
        
        # Tokenize and generate hashtags using the model
        inputs = tokenizer([text], max_length=512, truncation=True, return_tensors="pt")
        output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        tags = list(set(decoded_output.strip().split(", ")))  # Get unique hashtags
        return tags
    
    # Apply the generate_tags function to each row of the text column
    df['hashtag'] = df[text_column].apply(generate_tags)
    
    return df

# Generate hashtags for the 'text' column





In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk

def hashtag_generator(text):
    nltk.download('punkt')

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-base-tag-generation")
    model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-base-tag-generation")
    
    # Function to generate hashtags for a single text
    def generate_tags(text):
        if not isinstance(text, str) or len(text.strip()) == 0:
            return []  # Return an empty list for empty or missing text
        
        # Tokenize and generate hashtags using the model
        inputs = tokenizer([text], max_length=512, truncation=True, return_tensors="pt")
        output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        tags = list(set(decoded_output.strip().split(", ")))  # Get unique hashtags
        return tags

    return generate_tags(text)


In [13]:
text = "our fans our players our summer this is your threelions squad announcement video for euro httpstcoqugnzgsp"
hashtags = hashtag_generator(text)
print(hashtags)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beaut\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['World', 'Europe', 'Football']


In [ ]:
sentiment_result = sentiment_analysis_transformers(df2, text='tweet_content')
df_with_hashtags = hashtag_generator(df2, text_column='tweet_content')
df_with_hashtags =  df_with_hashtags.drop(columns = ["scores"])
df_with_hashtags['id_dist'] = [uuid.uuid4() for _ in range(len(df_with_hashtags))]
df_with_hashtags['tweet_location'] = ""

d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Assign unique IDs


# Generate a unique ID for each row
df_with_hashtags['id_dist'] = [uuid.uuid4() for _ in range(len(df_with_hashtags))]
df_with_hashtags['tweet_location'] = ""


# df_with_hashtags['id_dist'] = range(1, len(df_with_hashtags) + 1)
# df_with_hashtags['tweet_location'] = ""

In [45]:
df2

,tweet_location,tweet_date,tweet_content
0,NYC,02-03-2020,trending new yorkers encounter empty supermark...
1,"Seattle, WA",02-03-2020,when i couldnt find hand sanitizer at fred mey...
2,NaN,02-03-2020,find out how you can protect yourself and love...
3,Chicagoland,02-03-2020,panic buying hits newyork city as anxious shop...
4,"Melbourne, Victoria",03-03-2020,toiletpaper dunnypaper coronavirus coronavirus...
...,...,...,...
3793,Israel ??,16-03-2020,meanwhile in a supermarket in israel people da...
3794,"Farmington, NM",16-03-2020,did you panic buy a lot of nonperishable items...
3795,"Haverford, PA",16-03-2020,asst prof of economics cconces was on nbcphila...
3796,NaN,16-03-2020,gov need to do somethings instead of biar je r...


In [47]:
import pandas as pd
import uuid
import numpy as np


# List to store processed DataFrames
processed_dfs = []

# Split the DataFrame into 5 equal parts
split_dfs = np.array_split(df2, 5)

for df_part in split_dfs:
    # Apply sentiment analysis
    sentiment_result = sentiment_analysis_transformers(df_part, text='tweet_content')
    
    # Apply hashtag generator and drop "scores" column
    df_with_hashtags = hashtag_generator(df_part, text_column='tweet_content')
    df_with_hashtags = df_with_hashtags.drop(columns=["scores"])
    
    # Add unique IDs and set empty 'tweet_location'
    df_with_hashtags['id_dist'] = [uuid.uuid4() for _ in range(len(df_with_hashtags))]
        
    # Append the processed DataFrame to the list
    processed_dfs.append(df_with_hashtags)

# Concatenate all processed DataFrames into a single DataFrame
final_df = pd.concat(processed_dfs, ignore_index=True)


d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWa

In [48]:
final_df

,tweet_location,tweet_date,tweet_content,Sentiment,sentiment_score,hashtag,id_dist
0,NYC,02-03-2020,trending new yorkers encounter empty supermark...,Negative 😡,0.500374,"[Virus, New York City, Coronavirus]",1891c43a-d795-493a-9d35-dacc73992983
1,"Seattle, WA",02-03-2020,when i couldnt find hand sanitizer at fred mey...,Negative 😡,0.874354,"[Virus, Covid, Covid 19, Coronavirus]",90453505-3e57-4f75-9347-8df4a49ec6a1
2,NaN,02-03-2020,find out how you can protect yourself and love...,Positive ❤️,0.800672,"[Virus, Covid, Covid 19, Coronavirus]",b2ad8e08-c504-428a-a8eb-319379e94022
3,Chicagoland,02-03-2020,panic buying hits newyork city as anxious shop...,Negative 😡,0.898224,"[Virus, New York City, Coronavirus]",39301481-881a-4bab-8556-cc263a12ee93
4,"Melbourne, Victoria",03-03-2020,toiletpaper dunnypaper coronavirus coronavirus...,Neutral 💛,0.584825,"[Covid 19, Self, Health, Virus, Mental Health,...",5b8f2f04-fe2d-4920-b4a3-b986d2f91985
...,...,...,...,...,...,...,...
3793,Israel ??,16-03-2020,meanwhile in a supermarket in israel people da...,Neutral 💛,0.664129,"[Virus, Covid, Covid 19, Coronavirus]",a7be15cb-2e1e-4e8e-a1e0-816e09cf73d1
3794,"Farmington, NM",16-03-2020,did you panic buy a lot of nonperishable items...,Neutral 💛,0.678648,"[Food, Volunteering, Covid 19, Foodies, Covid ...",46693621-5500-4cbb-9d6b-34d999c6b717
3795,"Haverford, PA",16-03-2020,asst prof of economics cconces was on nbcphila...,Neutral 💛,0.745936,"[Economics, Virus, Economy, Tech, Technology, ...",c9ac011a-8005-47e9-9333-cb51adc4d03f
3796,NaN,16-03-2020,gov need to do somethings instead of biar je r...,Negative 😡,0.919379,"[Covid 19, Virus, Covid 19 Covid 19 Covid 19 C...",c000158e-ba97-4552-8627-33bda0ab371a


In [49]:
# Set a column as index, then reset the index
df_with_hashtags = final_df.set_index('id_dist')  # Set 'id_dist' as the index temporarily

# Reset the index without the 'id_dist' column
df_with_hashtags = df_with_hashtags.reset_index(drop=False)

# Move 'id_dist' to the end of the DataFrame
df_with_hashtags = df_with_hashtags[['tweet_content', 'tweet_date', 'tweet_location', 'hashtag', 'sentiment_score', 'Sentiment', 'id_dist']]


In [ ]:
df_with_hashtags

In [57]:
# Explode the hashtags column
final_1 = df_with_hashtags.explode('hashtag')

# Reset the index for a cleaner output
final = final_1.reset_index(drop=True)
final.head()

,tweet_content,tweet_date,tweet_location,hashtag,sentiment_score,Sentiment,id_dist
0,trending new yorkers encounter empty supermark...,02-03-2020,NYC,Virus,0.500374,Negative 😡,1891c43a-d795-493a-9d35-dacc73992983
1,trending new yorkers encounter empty supermark...,02-03-2020,NYC,New York City,0.500374,Negative 😡,1891c43a-d795-493a-9d35-dacc73992983
2,trending new yorkers encounter empty supermark...,02-03-2020,NYC,Coronavirus,0.500374,Negative 😡,1891c43a-d795-493a-9d35-dacc73992983
3,when i couldnt find hand sanitizer at fred mey...,02-03-2020,"Seattle, WA",Virus,0.874354,Negative 😡,90453505-3e57-4f75-9347-8df4a49ec6a1
4,when i couldnt find hand sanitizer at fred mey...,02-03-2020,"Seattle, WA",Covid,0.874354,Negative 😡,90453505-3e57-4f75-9347-8df4a49ec6a1


In [ ]:
final

In [3]:
result1 = pd.read_csv("result.csv")
result1 = result1.drop(columns=result1.columns[0])  # Drop the first column
result1.dtypes

tweet_content       object
tweet_date          object
tweet_location     float64
hashtag             object
sentiment_score    float64
Sentiment           object
id_dist             object
dtype: object

In [18]:
result1['sentiment_score'] = result1['sentiment_score'].apply(lambda x: round(float(x), 2))
# Replace NaN with a default value if any are found
result1['sentiment_score'].fillna(0.0, inplace=True)
# Convert to float to ensure compatibility
result1['sentiment_score'] = result1['sentiment_score'].astype(float)
result1['tweet_location'] = result1['tweet_location'].astype(str)
result1.dtypes

C:\Users\beaut\AppData\Local\Temp\ipykernel_30016\1998775306.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result1['sentiment_score'].fillna(0.0, inplace=True)


tweet_content       object
tweet_date          object
tweet_location      object
hashtag             object
sentiment_score    float64
Sentiment           object
id_dist             object
dtype: object

In [10]:
result1['tweet_location'] = result1['tweet_location'].fillna('Unknown')  # or leave as None depending on your preference

# 3. Ensure sentiment_score is float (it should already be float64)
result1['sentiment_score'] = result1['sentiment_score'].astype(float)

# 4. Ensure id_dist is valid UUID (string)
result1['id_dist'] = result1['id_dist'].apply(lambda x: str(uuid.UUID(x)) if pd.notna(x) else None)

# 5. Ensure text columns are string type
result1['tweet_content'] = result1['tweet_content'].astype(str)
result1['hashtag'] = result1['hashtag'].astype(str)
result1['Sentiment'] = result1['Sentiment'].astype(str)


In [11]:
import pyodbc

# Connection string example
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8JEBTPG;'  # Specify the server name
    'DATABASE=tweets;'  # Specify the database name
    'UID=sa;'  # Specify your username
    'PWD=Triomphe&14;'  # Specify your password
)

# Create a cursor from the connection
cursor = conn.cursor()

# SQL INSERT query
insert_query = """
INSERT INTO [dbo].[tweet] 
           ([tweet_content], [tweet_date], [tweet_location], [hashtag], [sentiment_score], [sentiment], [id_dist])
     VALUES (?, ?, ?, ?, ?, ?, ?)
"""

# Iterate over DataFrame rows and insert data into SQL table
for index, row in result1.iterrows():
    cursor.execute(insert_query,
                   row['tweet_content'], 
                   row['tweet_date'], 
                   row['tweet_location'], 
                   row['hashtag'], 
                   row['sentiment_score'], 
                   row['Sentiment'],
                   row['id_dist'])

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()




In [ ]:
result2 = pd.read_csv("result_2.csv")

result2['tweet_location'] = result2['tweet_location'].fillna('Unknown')  # or leave as None depending on your preference
result2['tweet_date'] = result2['tweet_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# 3. Ensure sentiment_score is float (it should already be float64)
result2['sentiment_score'] = result2['sentiment_score'].astype(float)

# 4. Ensure id_dist is valid UUID (string)
result2['id_dist'] = result2['id_dist'].apply(lambda x: str(uuid.UUID(x)) if pd.notna(x) else None)

# 5. Ensure text columns are string type
result2['tweet_content'] = result2['tweet_content'].astype(str)
result2['hashtag'] = result2['hashtag'].astype(str)
result2['Sentiment'] = result2['Sentiment'].astype(str)




In [ ]:
# Convert 'tweet_date' to datetime
result2['tweet_date'] = pd.to_datetime(result2['tweet_date'], errors='coerce')

# Fill any NaT values with a default date if needed (e.g., '1900-01-01')
result2['tweet_date'].fillna(pd.Timestamp('1900-01-01'), inplace=True)

# Verify the data type of 'tweet_date'
print(result2['tweet_date'].dtype)  # Should output datetime64[ns]
print(result2[['tweet_date']].head())

In [31]:
import numpy as np

# Ensure 'sentiment_score' is a valid float and handle NaN values by replacing them with np.nan
result2['sentiment_score'] = result2['sentiment_score'].fillna(np.nan).astype(float)

# Ensure 'tweet_date' is a valid datetime and handle any conversion issues
result2['tweet_date'] = pd.to_datetime(result2['tweet_date'], errors='coerce')

# Fill any missing values in 'tweet_location' with 'Unknown' (or None if you prefer)
result2['tweet_location'] = result2['tweet_location'].fillna('Unknown')


In [ ]:
import pyodbc
import pandas as pd

# Connection string example
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8JEBTPG;'  # Specify the server name
    'DATABASE=tweets;'  # Specify the database name
    'UID=sa;'  # Specify your username
    'PWD=Triomphe&14;'  # Specify your password
)

# Create a cursor from the connection
cursor = conn.cursor()

# SQL INSERT query
insert_query = """
INSERT INTO [dbo].[tweet] 
           ([tweet_content], [tweet_date], [tweet_location], [hashtag], [sentiment_score], [sentiment], [id_dist])
     VALUES (?, ?, ?, ?, ?, ?, ?)
"""


# Iterate over DataFrame rows and insert data into SQL table
for index, row in result2.iterrows():
    cursor.execute(insert_query,
                   row['tweet_content'], 
                   row['tweet_date'], 
                   row['tweet_location'], 
                   row['hashtag'], 
                   row['sentiment_score'], 
                   row['Sentiment'],
                   row['id_dist'])

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


In [ ]:
final.to_csv('result_2.csv', index=False)

In [22]:
import pandas as pd

df_Sent1 = pd.read_csv("result.csv")
df_Sent2 = pd.read_csv("result_2.csv")

In [ ]:
import pandas as pd
from transformers import pipeline



# Function to get the dominant emotion
def get_dominant_emotion(text):
    # Set up the emotion classifier pipeline
    classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=False)    
    result = classifier(text)
    return result[0]['label']  # Return only the dominant emotion label


In [9]:
import pandas as pd
from transformers import pipeline
classifier = pipeline("text-classification", model="tasinhoque/roberta-large-go-emotions", return_all_scores=False)
print(classifier("I have no idea about this issue"))

[{'label': 'confusion', 'score': 0.9057261943817139}]


In [ ]:
# Apply the function to each row in the DataFrame
df_Sent1['emotion'] = df_Sent1['tweet_content'].apply(get_dominant_emotion)

# Display the DataFrame with the new 'dominant_emotion' column
# print(df_Sent1)
df_Sent1.head(10)

In [25]:
# Apply the function to each row in the DataFrame
df_Sent2['emotion'] = df_Sent2['tweet_content'].apply(get_dominant_emotion)

# Display the DataFrame with the new 'dominant_emotion' column
print(df_Sent2)

                                           tweet_content  tweet_date  \
0      trending new yorkers encounter empty supermark...  02-03-2020   
1      trending new yorkers encounter empty supermark...  02-03-2020   
2      trending new yorkers encounter empty supermark...  02-03-2020   
3      when i couldnt find hand sanitizer at fred mey...  02-03-2020   
4      when i couldnt find hand sanitizer at fred mey...  02-03-2020   
...                                                  ...         ...   
19700  i and forestandpaper members are committed to ...  16-03-2020   
19701  i and forestandpaper members are committed to ...  16-03-2020   
19702  i and forestandpaper members are committed to ...  16-03-2020   
19703  i and forestandpaper members are committed to ...  16-03-2020   
19704  i and forestandpaper members are committed to ...  16-03-2020   

            tweet_location           hashtag  sentiment_score    Sentiment  \
0                      NYC             Virus         0.50

In [29]:


# Convert 'tweet_date' to datetime
df_Sent2['tweet_date'] = pd.to_datetime(df_Sent2['tweet_date'], format='%d-%m-%Y')

# Normalize the datetime values to set time to 00:00:00
df_Sent2['tweet_date'] = df_Sent2['tweet_date'].apply(lambda x: x.normalize())



In [30]:
df_Sent1.to_csv('result_emotion_1.csv', index=False)
df_Sent2.to_csv('result_emotion_2.csv', index=False)

In [14]:
import pandas as pd

df_Sent2 = pd.read_csv("result_emotion_2.csv")
df_Sent1 = pd.read_csv("result_emotion_1.csv")

In [ ]:
df_Sent1

In [ ]:
df_Sent2['tweet_location'] = df_Sent2['tweet_location'].fillna('Unknown')
df_Sent2['tweet_date'] = pd.to_datetime(df_Sent2['tweet_date'], errors='coerce')
df_Sent2['sentiment_score'] = df_Sent2['sentiment_score'].fillna(0).round(2) 

In [17]:
import pyodbc
import pandas as pd

# Connection string example
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8JEBTPG;'  # Specify the server name
    'DATABASE=tweets;'  # Specify the database name
    'UID=sa;'  # Specify your username
    'PWD=Triomphe&14;'  # Specify your password
)

# Create a cursor from the connection
cursor = conn.cursor()

# SQL INSERT query
insert_query = """
INSERT INTO [dbo].[tweet] 
           ([tweet_content], [tweet_date], [tweet_location], [hashtag], [sentiment_score], [sentiment], [id_dist], [emotion])
     VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""


# Now try inserting the data again
try:
    for index, row in df_Sent2.iterrows():
        cursor.execute(insert_query,
                       row['tweet_content'], 
                       row['tweet_date'], 
                       row['tweet_location'], 
                       row['hashtag'], 
                       row['sentiment_score'], 
                       row['Sentiment'],
                       row['id_dist'],
                       row['emotion'])
    conn.commit()  # Commit if no exceptions occur
except Exception as e:
    print(f"Error during insertion: {e}")

# Finally, close the cursor and connection
cursor.close()
conn.close()


Error during insertion: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 6 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')


In [44]:
import pyodbc
import pandas as pd

# Configuration de la connexion SQL Server
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-8JEBTPG;'
    'DATABASE=tweets;'
    'UID=sa;'
    'PWD=Triomphe&14;'
)

cursor = conn.cursor()

# SQL INSERT query
insert_query = """
INSERT INTO [dbo].[tweet] 
           ([tweet_content], [tweet_date], [tweet_location], [hashtag], [sentiment_score], [sentiment], [id_dist], [emotion])
     VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

# Assurer que `sentiment_score` est bien float64 et sans NaN
df_Sent2['sentiment_score'] = pd.to_numeric(df_Sent2['sentiment_score'], errors='coerce')
df_Sent2['sentiment_score'] = df_Sent2['sentiment_score'].fillna(0).astype('float64')  # Remplace NaN par 0 si nécessaire

# Tentative d'insertion avec gestion d'erreurs détaillée
try:
    for index, row in df_Sent2.iterrows():
        try:
            # Exécutez la commande d'insertion pour chaque ligne
            cursor.execute(insert_query,
                           row['tweet_content'], 
                           row['tweet_date'], 
                           row['tweet_location'], 
                           row['hashtag'], 
                           row['sentiment_score'],  # Assurez-vous que c'est un float64
                           row['Sentiment'],
                           row['id_dist'],
                           row['emotion'])
        except Exception as e:
            print(f"Erreur lors de l'insertion à l'index {index} : {e}")
    conn.commit()  # Commit si aucune exception
except Exception as e:
    print(f"Erreur globale pendant l'insertion : {e}")

# Fermeture de la connexion
cursor.close()
conn.close()


Erreur lors de l'insertion à l'index 1842 : ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 7 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')
Erreur lors de l'insertion à l'index 6648 : ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 7 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')
Erreur lors de l'insertion à l'index 14274 : ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Serv